<a href="https://colab.research.google.com/github/luizthompson/MVP_Sprints/blob/main/MVP_Sprint_Machine_Learning_%26_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Sprint: Machine Learning & Analytics
Luiz Guilherme Thompson Vaz


# Definição do Problema

Descrição do Problema:
O problema em questão é analisar os resultados de partidas da Copa Libertadores da América, considerando informações como a edição do torneio, a rodada, a data da partida, os clubes envolvidos e os placares.

# Premissas ou Hipóteses


Os resultados das partidas refletem o desempenho das equipes.
As características dos clubes (força, estratégias, histórico, etc.) influenciam os resultados.
O formato do torneio permaneceu constante ao longo das edições analisadas.

# Restrições ou Condições para Selecionar os Dados

Os dados são confiáveis e abrangem várias edições da Copa Libertadores.
Os dados contém informações relevantes para análise, como os clubes envolvidos, as datas e os placares das partidas.

# Descrição do Dataset
O dataset consiste em um conjunto de dados tabulares com as seguintes colunas:

Edition: Ano da edição da Copa Libertadores.
Round: Rodada da partida (ex: Final, Semifinal, Quartas de Final, etc.).
Date: Data da partida.
Home Club: Clube mandante.
Away Club: Clube visitante.
Home Score: Placar do clube mandante.
Away Score: Placar do clube visitante.


In [19]:
# Importando as bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Carregar o dataset
url = 'https://raw.githubusercontent.com/luizthompson/MVP_Sprints/8dd5432382946862109f439d0a49642b781a45e7/libertadores-results-ds.csv'
df = pd.read_csv(url)

# Exibindo as primeiras linhas do dataset
print(df.head())



   Edition      Round        Date      Home Club      Away Club  Home Score  \
0     2023      Final   4/11/2023  Fluminense FC   Boca Juniors           2   
1     2023  Semifinal   6/10/2023      Palmeiras   Boca Juniors           1   
2     2023  Semifinal   5/10/2023  Internacional  Fluminense FC           1   
3     2023  Semifinal  29/09/2023   Boca Juniors      Palmeiras           0   
4     2023  Semifinal  28/09/2023  Fluminense FC  Internacional           2   

   AwayScore  
0          1  
1          1  
2          2  
3          0  
4          2  


# Preparação de Dados

Separação do Dataset
O dataset pode ser separado em conjuntos de treino e teste, onde os dados de uma determinada edição podem ser utilizados para treino e os dados de uma edição posterior podem ser usados para teste.


In [ ]:
# Splitting the dataset into train and test
train, test = train_test_split(df, test_size=0.2, random_state=42)

print(f'Size of training set: {len(train)}')
print(f'Size of testing set: {len(test)}')

Size of training set: 1456
Size of testing set: 365


# Validação Cruzada
Neste caso, temos um conjunto de dados de tamanho razoável (1456 amostras de treinamento e 365 amostras de teste). Isso é suficiente para treinar nosso modelo e ter uma estimativa confiável de seu desempenho em dados não vistos.Dado o tamanho do nosso conjunto de dados, a divisão única de treinamento/teste que estamos usando deve fornecer uma avaliação suficientemente precisa do desempenho do nosso modelo.Portanto, decidimos não usar a validação cruzada neste projeto para economizar recursos computacionais sem sacrificar a precisão da nossa avaliação do modelo.


# Transformações de Dados

Neste caso, foi realizado duas transformações principais:

1. Codificação One-Hot: As variáveis categóricas no conjunto de dados, que podem incluir características como o round do jogo e os clubes em casa e fora, são transformadas em uma forma que pode ser fornecida aos algoritmos de machine learning para melhorar a precisão das previsões. A codificação one-hot é um processo pelo qual variáveis categóricas são convertidas em uma forma que poderia ser fornecida aos algoritmos de ML para melhorar a precisão das previsões.

2. Padronização: Após a codificação one-hot, as variáveis numéricas são padronizadas para terem média 0 e desvio padrão 1. Isso é especialmente importante para os placares das partidas, garantindo que eles estejam na mesma escala e não distorçam o modelo

In [ ]:
# Importando as bibliotecas necessárias
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Carregando o conjunto de dados
url = 'https://raw.githubusercontent.com/luizthompson/MVP_Sprints/8dd5432382946862109f439d0a49642b781a45e7/libertadores-results-ds.csv'
df = pd.read_csv(url)

# Separando as variáveis categóricas
cat_vars = df.select_dtypes(include='object').columns.tolist()

# Aplicando a codificação one-hot nas variáveis categóricas
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_vars = encoder.fit_transform(df[cat_vars])

# Criando um DataFrame com as variáveis codificadas
encoded_df = pd.DataFrame(encoded_vars, columns=encoder.get_feature_names_out(cat_vars))

# Removendo as variáveis categóricas originais do DataFrame
df.drop(columns=cat_vars, inplace=True)

# Concatenando o DataFrame original com o DataFrame codificado
df = pd.concat([df, encoded_df], axis=1)

# Agora, vamos padronizar as variáveis numéricas
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


# Modelagem e Treinamento

1.  Seleção do algoritmo - Foi usado o algoritmo Random Forest Regressor ele é baseado em árvores de decisão e funciona criando um “floresta” de árvores de decisão independentes durante o treinamento e produzindo a média das previsões produzidas pelas árvores individuais.

2.  Ajuste inicial dos hiperparâmetros: Inicialmente, vamos usar os hiperparâmetros padrão fornecidos pela implementação do RandomForestRegressor no sklearn. Isso inclui o uso de um número de estimadores (árvores de decisão) igual a 100.

3.  Treinamento do modelo: Vamos treinar o modelo usando o conjunto de treinamento. Se o modelo estiver sofrendo de underfitting (ou seja, não está se ajustando bem nem mesmo aos dados de treinamento), podemos tentar aumentar a complexidade do modelo, por exemplo, aumentando o número de estimadores ou reduzindo o parâmetro de regularização.

4.  Otimização dos hiperparâmetros: Se necessário, podemos otimizar os hiperparâmetros do modelo usando técnicas como a pesquisa em grade (GridSearch) ou a pesquisa aleatória (RandomSearch). Isso envolve a experimentação de diferentes combinações de hiperparâmetros para encontrar a combinação que produz o melhor desempenho no conjunto de validação.

5.  Métodos avançados: Dependendo do desempenho do nosso modelo inicial, podemos considerar o uso de métodos mais avançados ou complexos. Isso pode incluir o uso de diferentes algoritmos de aprendizado de máquina, a criação de um comitê de modelos (ensemble), ou o uso de técnicas de aprendizado profundo.





In [ ]:
# Importando as bibliotecas necessárias
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Definindo o modelo
reg = RandomForestRegressor(random_state=42)

# Treinando o modelo
reg.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = reg.predict(X_test)

# Avaliando o modelo
print(f'Erro Quadrático Médio: {mean_squared_error(y_test, y_pred)}')



Erro Quadrático Médio: 0.16599215554281552


# Avaliação de Resultados

1. Neste caso, estamos usando o EQM, que é uma métrica comum para avaliar modelos de regressão. O EQM mede a média dos erros quadrados entre as previsões e os valores reais.

2. Treinamento e teste do modelo: O modelo foi treinado com toda a base de treino e testado com a base de teste. Isso é importante para avaliar como o modelo se comporta com dados não vistos.

3. Análise dos resultados: Com base no EQM obtido, podemos dizer que o modelo tem um desempenho razoável, embora a interpretação do EQM dependa muito do contexto e da escala dos seus dados.

4. Comparação de modelos: No momento, temos apenas um modelo, o RandomForestRegressor. Se tivéssemos mais modelos, poderíamos comparar o EQM de cada modelo para escolher o melhor.

5. Melhor solução encontrada: AA melhor solução encontrada até agora é o modelo RandomForestRegressor com os hiperparâmetros padrão.

6. Como o desempenho do modelo no conjunto de treinamento e no conjunto de teste é semelhante, não há indicações claras de overfitting. Overfitting ocorre quando um modelo se ajusta muito bem aos dados de treinamento, mas não generaliza bem para novos dados. No entanto, sem mais informações, como o desempenho do modelo no conjunto de treinamento, não podemos confirmar se há overfitting.


In [ ]:
# Importando as bibliotecas necessárias
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Definindo o modelo
reg = RandomForestRegressor(random_state=42)

# Treinando o modelo com toda a base de treino
reg.fit(X_train, y_train)

# Testando o modelo com a base de teste
y_pred = reg.predict(X_test)

# Avaliando o modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Erro Quadrático Médio: {mse}')


Erro Quadrático Médio: 0.16599215554281552


In [ ]:
# Importando as bibliotecas necessárias
from sklearn.linear_model import LinearRegression

# Definindo o modelo
reg = LinearRegression()

# Treinando o modelo
reg.fit(X_train, y_train)

# Fazendo previsões no conjunto de teste
y_pred = reg.predict(X_test)

# Avaliando o modelo
mse = mean_squared_error(y_test, y_pred)
print(f'Erro Quadrático Médio para a Regressão Linear: {mse}')


Erro Quadrático Médio para a Regressão Linear: 31.015735397380652
